# Практическое задание 3

# Named Entity Recognition

## Введение

### Постановка задачи

В этом задании вы будете решать задачу извлечения именованных сущностей (Named Entity Recognition) - одну из самых распространенных в NLP наряду с задачей текстовой классификации.

Данная задача заключается в том, что нужно классифицировать каждое слово / токен на предмет того, является ли оно частью именованной сущности (сущность может состоять из нескольких слов / токенов) или нет.

Например, мы хотим извлечь имена и названия организаций. Тогда для текста

    Yan    Goodfellow  works  for  Google  Brain

модель должна извлечь следующую последовательность:

    B-PER  I-PER       O      O    B-ORG   I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система (*BIO*-разметка) введена, чтобы различать последовательные именованные сущности одного типа.
Существуют и другие типы разметок, например *BILUO*, но в рамках данного практического задания сфокусируемся имеено на *BIO*.

Решать NER задачу мы будем на датасете CoNLL-2003 с использованием рекуррентных сетей и моделей на базе архитектуры Transformer.

### Библиотеки

Основные библиотеки:
 - [PyTorch](https://pytorch.org/)
 - [Transformers](https://github.com/huggingface/transformers)
 
### Данные

Данные лежат в архиве, который состоит из:

- *train.tsv* - обучающая выборка. В каждой строке записаны: <слово / токен>, <тэг слова / токена>

- *valid.tsv* - валидационная выборка, которую можно использовать для подбора гиперпарамеров и замеров качества. Имеет идентичную с train.tsv структуру.

- *test.tsv* - тестовая выборка, по которой оценивается итоговое качество. Имеет идентичную с train.tsv структуру.

Скачать данные можно здесь: [ссылка](https://github.com/dayyass/msu_task_3_ner)

In [1]:
!pip install numpy scikit-learn tensorboard torch tqdm transformers pytorch-crf

In [1]:
import random
from collections import Counter
from typing import Tuple, List, Dict, Any
from itertools import chain

import torch
import numpy as np

In [2]:
for d_id in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(d_id))
DEVICE = torch.device('cuda:0')
torch.cuda.is_available()

NVIDIA A100-PCIE-40GB
NVIDIA A100-PCIE-40GB
NVIDIA A100-PCIE-40GB
NVIDIA A100-PCIE-40GB


True

In [3]:
# download data in original segmentation
!rm -rd conll03
!mkdir conll03
!wget https://data.deepai.org/conll2003.zip -P conll03

--2022-11-21 19:00:10--  https://data.deepai.org/conll2003.zip
Resolving data.deepai.org (data.deepai.org)... 5.9.140.253
Connecting to data.deepai.org (data.deepai.org)|5.9.140.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982975 (960K) [application/x-zip-compressed]
Saving to: ‘conll03/conll2003.zip’

conll2003.zip       100%[===================>] 959.94K  4.23MB/s    in 0.2s    

2022-11-21 19:00:11 (4.23 MB/s) - ‘conll03/conll2003.zip’ saved [982975/982975]



In [4]:
!unzip -o conll03/conll2003.zip -d conll03/data
!ls conll03/data

Archive:  conll03/conll2003.zip
  inflating: conll03/data/metadata   
  inflating: conll03/data/test.txt   
  inflating: conll03/data/train.txt  
  inflating: conll03/data/valid.txt  
metadata  test.txt  train.txt  valid.txt


In [5]:
# download pretrained word embeddings
!rm -rd glove
!mkdir glove
!wget http://nlp.stanford.edu/data/glove.6B.zip -P glove

--2022-11-21 19:00:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-21 19:00:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-21 19:00:13--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove/glove.6B.zip’

In [6]:
!unzip -o glove/glove*.zip -d glove/data
!ls glove/data

Archive:  glove/glove.6B.zip
  inflating: glove/data/glove.6B.50d.txt  
  inflating: glove/data/glove.6B.100d.txt  
  inflating: glove/data/glove.6B.200d.txt  
  inflating: glove/data/glove.6B.300d.txt  
glove.6B.100d.txt  glove.6B.200d.txt  glove.6B.300d.txt  glove.6B.50d.txt


Зафиксируем seed для воспроизводимости результатов (желательно делать **всегда**!):

In [7]:
def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_global_seed(42)

## Часть 1. Подготовка данных (4 балла)

Первым делом нам нужно считать данные. Давайте напишем функцию, которая на вход принимает путь до одного из conll-2003 файла и возвращает два списка:
- список списков слов / токенов (и соответствующий ему)
- список списков тегов

P.S. Сделаем данную функцию более гибкой, подавая на вход еще булеву переменную, считываем ли мы данные в *lowercase* или нет.

**Задание. Реализуйте функцию read_conll2003.** **<font color='red'>(1 балл)</font>**

In [8]:
from io import StringIO
from typing import Iterable, Optional
from dataclasses import dataclass
from pathlib import Path


@dataclass
class Document:
    sentences: List[List[str]]
    labels: List[List[str]]


def read_conll2003(path: Path, lower: bool = True, convert_to_iob: bool = False) -> List[Document]:
    """
    Prepares data in CoNNL like format.
    """

    def entity_type(entity_label: str) -> str:
        return entity_label.split('-')[-1]

    prev_label = 'O'
    def convert(lbl: str) -> str:
        """In IOB scheme we mark the beginning of a new entity if and only if it follows the entity of the same type"""
        if lbl.startswith('B'):
            curr_type = entity_type(lbl)
            if prev_label == 'O' or entity_type(prev_label) != curr_type:
                lbl = 'I-' + curr_type
        return lbl


    preprocess_token = str.lower if lower else lambda x: x
    preprocess_label = convert if convert_to_iob else lambda x: x

    def read_lines(f: StringIO) -> Iterable[Tuple[Tuple[str, str], bool, bool]]:
        """(word, label), new_sentence, new_doc"""
        nonlocal prev_label

        new_sentence = True
        new_doc = True
        for line in f:
            line = line.strip()

            if not len(line):
                new_sentence = True
                prev_label = 'O'
                continue

            if line == '-DOCSTART- -X- -X- O':
                new_doc = True
                new_sentence = True
                prev_label = 'O'
                continue

            token_info = line.split()
            info_token, info_label = token_info[0], token_info[-1]
            yield (preprocess_token(info_token), preprocess_label(info_label)), new_sentence, new_doc

            new_sentence = False
            new_doc = False
            prev_label = info_label

    curr_doc: Optional[Document] = None
    curr_sentence: Optional[List[str]] = None
    curr_labels: Optional[List[str]] = None

    docs: List[Document] = []

    with open(path) as dataset_file:
        for (token, label), create_new_sentence, create_new_doc in read_lines(dataset_file):
            if create_new_sentence or create_new_doc:
                if curr_sentence is not None and len(curr_sentence):
                    curr_doc.sentences.append(curr_sentence)
                    curr_doc.labels.append(curr_labels)
                curr_sentence = []
                curr_labels = []

            if create_new_doc:
                if curr_doc is not None and len(curr_doc.sentences):
                    docs.append(curr_doc)
                curr_doc = Document([], [])

            curr_sentence.append(token)
            curr_labels.append(label)

        if curr_doc is None:
            raise ValueError(f'{path} dataset is empty!')

        if len(curr_sentence):
            curr_doc.sentences.append(curr_sentence)
            curr_doc.labels.append(curr_labels)

        if len(curr_doc.sentences):
            docs.append(curr_doc)

    print(f'Read {len(docs)} documents and {sum(map(lambda d: len(d.sentences), docs))} sentences from {path}')
    return docs

Считаем все три файла:
- *train.txt*
- *valid.txt*
- *test.txt*

In [9]:
train_docs = read_conll2003(Path('conll03/data/train.txt'))
val_docs = read_conll2003(Path('conll03/data/valid.txt'))
test_docs = read_conll2003(Path('conll03/data/test.txt'))

Read 946 documents and 14041 sentences from conll03/data/train.txt
Read 216 documents and 3250 sentences from conll03/data/valid.txt
Read 231 documents and 3453 sentences from conll03/data/test.txt


Посмотрим на то, что мы получили:

In [10]:
for token, label in zip(train_docs[0].sentences[0], train_docs[0].labels[0]):
    print(f"{token}\t{label}")

eu	B-ORG
rejects	O
german	B-MISC
call	O
to	O
boycott	O
british	B-MISC
lamb	O
.	O


In [11]:
for token, label in zip(val_docs[0].sentences[0], val_docs[0].labels[0]):
    print(f"{token}\t{label}")

cricket	O
-	O
leicestershire	B-ORG
take	O
over	O
at	O
top	O
after	O
innings	O
victory	O
.	O


In [12]:
for token, label in zip(test_docs[0].sentences[0], test_docs[0].labels[0]):
    print(f"{token}\t{label}")

soccer	O
-	O
japan	B-LOC
get	O
lucky	O
win	O
,	O
china	B-PER
in	O
surprise	O
defeat	O
.	O


In [13]:
assert all(len(train_doc.sentences) == len(train_doc.labels) for train_doc in train_docs), "Длины тренировочных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert all(len(val_doc.sentences) == len(val_doc.labels) for val_doc in val_docs), "Длины валидационных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert all(len(test_doc.sentences) == len(test_doc.labels) for test_doc in test_docs), "Длины тестовых token_seq и label_seq не совпадают, ошибка в функции read_conll2003"

assert train_docs[0].sentences[0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Ошибка в тренировочном token_seq"
assert train_docs[0].labels[0] == ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'], "Ошибка в тренировочном label_seq"

assert val_docs[0].sentences[0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Ошибка в валидационном token_seq"
assert val_docs[0].labels[0] == ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], "Ошибка в валидационном label_seq"

assert test_docs[0].sentences[0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Ошибка в тестовом token_seq"
assert test_docs[0].labels[0] == ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'], "Ошибка в тестовом label_seq"

print("Тесты пройдены!")

Тесты пройдены!


Датасет CoNLL-2003 представлен в виде разметки **BIO**, где лейбл:
- *B-{label}* - начало сущности *{label}*
- *I-{label}* - продолжение сущности *{label}*
- *O* - отсутсвие сущности

Также существует другие разметки последовательностей, например **BILUO**. Подробнее с разметками можно ознакомится во вспомогательном ноутбуке.

In [14]:
# read glove embeddings
def read_glove(path: Path) -> Dict[str, np.ndarray]:
    print(f'Extracting word embeddings from {path}')

    embeddings_index = {}
    with open(path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print(f'Extracted {len(embeddings_index)} embeddings')
    return embeddings_index


def glove2vocab(path: Path) -> Dict[str, int]:
    print(f'Building pretrained vocabulary from {path}')

    vocab = {}
    with open(path, encoding='utf-8') as f:
        for word_idx, line in enumerate(f):
            values = line.split()
            word = values[0]
            vocab[word] = word_idx

    print(f'Vocabulary size: {len(vocab)}')
    return vocab

In [15]:
from pprint import pprint
pprint(dict(list(read_glove(Path('glove/data/glove.6B.50d.txt')).items())[399990:400000]))

Extracting word embeddings from glove/data/glove.6B.50d.txt
Extracted 400000 embeddings
{'1.3775': array([-0.24171  , -0.23367  ,  0.10672  , -1.6023   ,  0.1244   ,
       -0.016423 ,  0.1302   ,  0.70318  , -0.14301  ,  0.47307  ,
       -0.67426  , -0.73478  ,  0.92795  , -0.2342   ,  0.1281   ,
       -0.3318   ,  0.084984 , -1.0039   , -0.81191  ,  0.67333  ,
        1.0034   ,  0.067612 , -0.67016  , -0.60906  , -0.018144 ,
        0.34908  ,  0.41337  ,  0.043822 ,  0.22416  , -0.89374  ,
       -1.5504   ,  0.7269   ,  0.41673  ,  0.34538  , -0.22609  ,
        0.41702  ,  0.80483  , -0.097597 , -0.67301  , -0.30269  ,
        0.77956  , -0.072954 ,  0.16443  , -0.0061933,  0.061141 ,
       -0.28784  ,  0.58601  ,  0.47279  , -0.61084  , -0.72091  ],
      dtype=float32),
 'aqm': array([-1.1167e+00,  1.4057e-01,  3.6302e-01, -1.3836e-01, -1.4797e+00,
       -9.8573e-01,  4.0487e-01, -3.9773e-01, -4.0102e-01,  3.4691e-01,
        3.8857e-01,  2.9772e-01,  8.2807e-01, -2.4541e-0

### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения:
- {**token**}→{**token_idx**}: соответствие между словом / токеном и строкой в *embedding* матрице (начинается с 0);
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0);

Теперь нам необходимо реализовать две функции:
- get_token2idx
- get_label2idx

которые будут возвращать соответствующие словари.

P.S. token2idx словарь должен также содержать специальные токены:
- `<PAD>` - спецтокен для паддинга, так как мы собираемся обучать модели батчами
- `<UNK>` - спецтокен для обработки слов / токенов, которых нет в словаре (актуально для инференса)

Давайте для удобства дадим им idx 0 и 1 соответственно.

P.P.S. В get_token2idx можно также добавить параметр *min_count*, который будет включать только слова превышающие определенную частоту.

Сначала соберем:
- token2cnt - словарь из уникального слова / токена в количество это слова / токена в тренировочной выборке (важно, что только в тренировочной!)
- label_set - список из уникальных тегов

P.S. Также можно использовать стемминг для того, чтобы преобразовывать разные словоформы одного слова в один токен, но мы опустим этот момент.

**Задание. Реализуйте функции get_token2idx и get_label2idx.** **<font color='red'>(1 балл)</font>**

In [16]:
def get_token_counts(docs: Iterable[Document], lower: bool = True):
    postprocess = (lambda tok: tok.lower()) if lower else (lambda x: x)
    return Counter([postprocess(token) for doc in docs for sentence in doc.sentences for token in sentence])

token2cnt = get_token_counts(train_docs, lower=False)

In [17]:
token2cnt.most_common(10)

[('the', 8390),
 ('.', 7374),
 (',', 7290),
 ('of', 3815),
 ('in', 3621),
 ('to', 3424),
 ('a', 3199),
 ('and', 2872),
 ('(', 2861),
 (')', 2861)]

In [18]:
print(f"Количество уникальных слов в тренировочном датасете: {len(token2cnt)}")
print(f"Количество слов встречающихся только один раз в тренировочном датасете: {len([token for token, cnt in token2cnt.items() if cnt == 1])}")

Количество уникальных слов в тренировочном датасете: 21009
Количество слов встречающихся только один раз в тренировочном датасете: 10060


Как мы видим, у нас есть много слов, которые встречаются только один раз в датасете. Очевидно, что выучиться по ним у нас не получиться, мы только переобучимся, поэтому давайте выкинем такие слова при формировании нашего словаря.

In [19]:
# используйте параметр min_count для того, чтобы отсекать слова частотой cnt < min_count

def get_token2idx(token_counts: Dict[str, int], min_count: int, base_dict: Optional[Dict[str, int]] = None) -> Dict[str, int]:
    """
    Get mapping from tokens to indices to use with Embedding layer.
    """
    if base_dict is None:
        result = {'<PAD>': 0, '<UNK>': 1}
        curr_idx = 2
    else:
        result = deepcopy(base_dict)
        curr_idx = len(result)
        if '<PAD>' not in result:
            result['<PAD>'] = curr_idx
            curr_idx += 1
        if '<UNK>' not in result:
            result['<UNK>'] = curr_idx
            curr_idx += 1

    for tok in filter(lambda t: t not in result, token_counts.keys()):
        tok_count = token_counts[tok]
        if tok_count < min_count:
            continue

        result[tok] = curr_idx
        curr_idx += 1

    print(f'Final vocabulary size: {len(result)}')
    return result


def get_char2idx(tokens: Iterable[str], num_embedding: bool = False) -> Dict[str, int]:
    result = {'<PAD>': 0, '<UNK>': 1}
    curr_idx = 3 if num_embedding else 2
    for char in set(chain.from_iterable(tokens)):
        if char.isdecimal() and num_embedding:
            result[char] = 2
        else:
            result[char] = curr_idx
            curr_idx += 1

    return result

In [20]:
t2idx = get_token2idx(token2cnt, min_count=2)
len(t2idx)

Final vocabulary size: 10951


10951

In [21]:
c2idx = get_char2idx(token2cnt.keys())
c2idx

{'<PAD>': 0,
 '<UNK>': 1,
 '0': 2,
 '!': 3,
 '1': 4,
 ']': 5,
 'u': 6,
 '$': 7,
 't': 8,
 '7': 9,
 'm': 10,
 '=': 11,
 '3': 12,
 'h': 13,
 '2': 14,
 '@': 15,
 '5': 16,
 '+': 17,
 '8': 18,
 'j': 19,
 '9': 20,
 "'": 21,
 'f': 22,
 '`': 23,
 ',': 24,
 'a': 25,
 '4': 26,
 ';': 27,
 '*': 28,
 '-': 29,
 '[': 30,
 'd': 31,
 'g': 32,
 'e': 33,
 ':': 34,
 '?': 35,
 'k': 36,
 'y': 37,
 '&': 38,
 '%': 39,
 'l': 40,
 'i': 41,
 ')': 42,
 'c': 43,
 'w': 44,
 'n': 45,
 '"': 46,
 '(': 47,
 'v': 48,
 '.': 49,
 'q': 50,
 's': 51,
 'o': 52,
 '6': 53,
 '/': 54,
 'r': 55,
 'x': 56,
 'z': 57,
 'p': 58,
 'b': 59}

In [22]:
get_char2idx(token2cnt.keys(), num_embedding=True)

{'<PAD>': 0,
 '<UNK>': 1,
 '0': 2,
 '!': 3,
 '1': 2,
 ']': 4,
 'u': 5,
 '$': 6,
 't': 7,
 '7': 2,
 'm': 8,
 '=': 9,
 '3': 2,
 'h': 10,
 '2': 2,
 '@': 11,
 '5': 2,
 '+': 12,
 '8': 2,
 'j': 13,
 '9': 2,
 "'": 14,
 'f': 15,
 '`': 16,
 ',': 17,
 'a': 18,
 '4': 2,
 ';': 19,
 '*': 20,
 '-': 21,
 '[': 22,
 'd': 23,
 'g': 24,
 'e': 25,
 ':': 26,
 '?': 27,
 'k': 28,
 'y': 29,
 '&': 30,
 '%': 31,
 'l': 32,
 'i': 33,
 ')': 34,
 'c': 35,
 'w': 36,
 'n': 37,
 '"': 38,
 '(': 39,
 'v': 40,
 '.': 41,
 'q': 42,
 's': 43,
 'o': 44,
 '6': 2,
 '/': 45,
 'r': 46,
 'x': 47,
 'z': 48,
 'p': 49,
 'b': 50}

In [23]:
max(map(len, token2cnt.keys())), sum(map(len, token2cnt.keys())) / len(token2cnt)

(61, 6.882716930839164)

In [24]:
# Функция для сортировки тегов, чтобы сначала был тег O, потом теги B- и только после теги I- (можно задать вручную)

def sort_labels_func(x: str) -> int:
    if x == "O":
        return 0
    elif x.startswith("B-"):
        return 1
    else:
        return 2

label_set = sorted(
    set(label for doc in train_docs for sentence_labels in doc.labels for label in sentence_labels),
    key=lambda x: (sort_labels_func(x), x),
)

In [25]:
label_set

['O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']

In [26]:
def get_label2idx(labels: List[str]) -> Dict[str, int]:
    """
    Get mapping from labels to indices.
    """

    result: Dict[str, int] = {}

    for label_idx, label in enumerate(labels):
        result[label] = label_idx

    return result

In [27]:
label2idx = get_label2idx(label_set)
label2idx

{'O': 0,
 'B-LOC': 1,
 'B-MISC': 2,
 'B-ORG': 3,
 'B-PER': 4,
 'I-LOC': 5,
 'I-MISC': 6,
 'I-ORG': 7,
 'I-PER': 8}

Посмотрим на то, что мы получили:

In [28]:
for token, idx in list(t2idx.items())[:10]:
    print(f"{token}\t{idx}")

<PAD>	0
<UNK>	1
eu	2
german	3
call	4
to	5
boycott	6
british	7
lamb	8
.	9


In [29]:
for label, idx in label2idx.items():
    print(f"{label}\t{idx}")

O	0
B-LOC	1
B-MISC	2
B-ORG	3
B-PER	4
I-LOC	5
I-MISC	6
I-ORG	7
I-PER	8


In [30]:
assert len(get_token2idx(token2cnt, min_count=1)) == 21011, "Ошибка в длине словаря, скорее всего неверно реализован min_count"
assert len(t2idx) == 10951, "Неправильная длина token2idx, скорее всего неверно реализован min_count"
assert len(label2idx) == 9, "Неправильная длина label2idx"

assert list(t2idx.items())[:10] == [('<PAD>', 0), ('<UNK>', 1), ('eu', 2), ('german', 3), ('call', 4), ('to', 5), ('boycott', 6), ('british', 7), ('lamb', 8), ('.', 9)], "Неправильно сформированный token2idx"
assert label2idx == {'O': 0, 'B-LOC': 1, 'B-MISC': 2, 'B-ORG': 3, 'B-PER': 4, 'I-LOC': 5, 'I-MISC': 6, 'I-ORG': 7, 'I-PER': 8}, "Неправильно сформированный label2idx"

print("Тесты пройдены!")

Final vocabulary size: 21011
Тесты пройдены!


### Подготовка датасета и загрузчика

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины.

Из предыдущего практического задания вы должны знать о `Dataset`'е (`torch.utils.data.Dataset`) - структура данных, которая хранит и может по индексу отдавать данные для обучения. Датасет должен наследоваться от стандартного PyTorch класса Dataset и переопределять методы `__len__` и `__getitem__`.

Метод `__getitem__` должен возвращать индексированную последовательность и её теги.

**Не забудьте** про `<UNK>` спецтокен для неизвестных слов!
    
Давайте напишем кастомный датасет под нашу задачу, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов
- token2idx
- label2idx

и возвращать из метода `__getitem__` два int64 тензора (`torch.LongTensor`) из индексов слов / токенов в сэмпле и индексов соответвующих тегов:

**Задание. Реализуйте класс датасета NERDataset.** **<font color='red'>(1 балл)</font>**

In [31]:
from functools import partial
from itertools import starmap, chain
from torch import LongTensor


@dataclass
class TokenizedDocument:
    char_ids: List[LongTensor]
    token_ids: List[LongTensor]
    label_ids: List[LongTensor]


def group_sentences(doc: TokenizedDocument, max_sequence_length: int) -> Iterable[Tuple[LongTensor, LongTensor, LongTensor]]:
    curr_length = 0
    grouped_chars: List[LongTensor] = []
    grouped_sentences: List[LongTensor] = []
    grouped_label_ids: List[LongTensor] = []

    for char_ids, sentence, label_ids in zip(doc.char_ids, doc.token_ids, doc.label_ids):
        if len(sentence) > max_sequence_length:
            print('Extra long sentence detected!!')

        if len(sentence) + curr_length < max_sequence_length or not len(grouped_sentences):
            grouped_chars.append(char_ids)
            grouped_sentences.append(sentence)
            grouped_label_ids.append(label_ids)
            curr_length += len(sentence)
            continue

        yield torch.concat(grouped_chars), torch.concat(grouped_sentences), torch.concat(grouped_label_ids)
        grouped_chars = [char_ids]
        grouped_sentences = [sentence]
        grouped_label_ids = [label_ids]
        curr_length = len(sentence)

    if len(grouped_sentences):
        yield torch.concat(grouped_chars), torch.concat(grouped_sentences), torch.concat(grouped_label_ids)


class NERDataset(torch.utils.data.Dataset):
    """
    PyTorch Dataset for NER.
    """

    def __init__(
            self,
            docs: Iterable[Document],
            char2idx: Dict[str, int],
            token2idx: Dict[str, int],
            label2idx: Dict[str, int],
            max_token_length: int,
            max_sequence_length: int,
            lower: bool
    ):
        self.char2idx = char2idx
        self.token2idx = token2idx
        self.label2idx = label2idx

        # group sentences into examples of max_sequence_length length

        def chars_processor(tokens: List[str]) -> LongTensor:
            return torch.stack(list(map(
                partial(self.process_chars, max_token_length=max_token_length, char2idx=self.char2idx),
                tokens
            ))).long()

        tokens_processor = partial(self.process_tokens, token2idx=self.token2idx, lower=lower)
        labels_processor = partial(self.process_labels, label2idx=self.label2idx)

        tokenized_docs = starmap(
            TokenizedDocument,
            map(lambda doc: (
                list(map(chars_processor, doc.sentences)),
                list(map(tokens_processor, doc.sentences)),
                list(map(labels_processor, doc.labels))
            ), docs)
        )

        sentence_grouper = partial(group_sentences, max_sequence_length=max_sequence_length)

        self._examples: List[Tuple[LongTensor, ...]] = list(chain.from_iterable(map(sentence_grouper, tokenized_docs)))

    def __len__(self):
        return len(self._examples)

    def __getitem__(self, idx: int) -> Tuple[LongTensor, ...]:
        return self._examples[idx]

    @staticmethod
    def process_chars(
            token: str,
            char2idx: Dict[str, int],
            max_token_length: int,
            pad: str = "<PAD>",
            unk: str = "<UNK>"
    ) -> LongTensor:
        """
        Transform list of tokens into list of tokens' indices.
        """
        unk_idx = char2idx[unk]
        chars = list(token[:max_token_length]) + (max_token_length - len(token)) * [pad]
        return torch.tensor(list(map(lambda tok: char2idx.get(tok, unk_idx), chars)), dtype=torch.long).long()
    
    @staticmethod
    def process_tokens(tokens: List[str], token2idx: Dict[str, int], unk: str = "<UNK>", lower: bool = False) -> LongTensor:
        """
        Transform list of tokens into list of tokens' indices.
        """
        preprocess = str.lower if lower else lambda x: x
        unk_idx = token2idx[unk]
        return torch.tensor(list(map(lambda tok: token2idx.get(preprocess(tok), unk_idx), tokens)), dtype=torch.long).long()

    @staticmethod
    def process_labels(labels: List[str], label2idx: Dict[str, int]) -> LongTensor:
        """
        Transform list of labels into list of labels' indices.
        """
        return  torch.tensor(list(map(label2idx.__getitem__, labels)), dtype=torch.long).long()

Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [32]:
train_ds = NERDataset(
    train_docs,
    char2idx=c2idx, token2idx=t2idx, label2idx=label2idx,
    max_token_length=20, max_sequence_length=512,
    lower=False
)
valid_ds = NERDataset(
    val_docs,
    char2idx=c2idx, token2idx=t2idx, label2idx=label2idx,
    max_token_length=20, max_sequence_length=512,
    lower=False
)
test_ds = NERDataset(
    test_docs,
    char2idx=c2idx, token2idx=t2idx, label2idx=label2idx,
    max_token_length=20, max_sequence_length=512,
    lower=False
)

Посмотрим на то, что мы получили:

In [33]:
train_ds[0]

(tensor([[33,  6,  0,  ...,  0,  0,  0],
         [55, 33, 19,  ...,  0,  0,  0],
         [32, 33, 55,  ...,  0,  0,  0],
         ...,
         [52, 48, 33,  ...,  0,  0,  0],
         [41, 10, 58,  ...,  0,  0,  0],
         [49,  0,  0,  ...,  0,  0,  0]]),
 tensor([  2,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
          15,  16,  17,  18,  19,  20,  21,  22,   3,  23,   5,  24,   5,   1,
           7,   8,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,   5,  35,
           9,  36,  37,  38,   5,  14,  15,  39,  37,  40,  41,  42,  43,  17,
          18,  44,  24,  45,  46,  47,  48,  49,  50,  51,  52,  25,  14,  53,
          23,  54,   1,   9,  55,  56,  57,  58,  59,  60,  61,  62,  63,  56,
          57,  58,  64,  60,  65,  66,  20,  67,  55,  14,  16,  37,  68,  69,
           1,  70,  71,   1,  72,  73,  74,  75,   9,  76,  17,  77,  53,  78,
          54,  79,  80,  81,  20,  54,  82,  83,  84,  54,  85,  20,  45,  33,
          86,  87,  14,  15

In [34]:
valid_ds[0]

(tensor([[43, 55, 41,  ...,  0,  0,  0],
         [29,  0,  0,  ...,  0,  0,  0],
         [40, 33, 41,  ...,  0,  0,  0],
         ...,
         [22, 52, 55,  ...,  0,  0,  0],
         [ 8, 13, 55,  ...,  0,  0,  0],
         [49,  0,  0,  ...,  0,  0,  0]]),
 tensor([ 1736,   570,  1776,   197,   686,   145,   348,   111,  1818,  1557,
             9,   247, 10678,   885,  4305,  2002,  1453,  1779,   678,  1891,
            66,  6269,    18,   968,   134,  1776,  1518,  1752,    87,   146,
          1818,    80,  3090,  2544,   215,   357,   840,     5,   197,   686,
           145,    14,  1568,   150,    14,  1737,  1738,     9,   184,  3881,
            18,   348,    67,  4631,    67,   941,    33,     1,   134,  1708,
           354,  1767,    67,  6509,    80,  1800,   377,  1198,   215,    18,
          1557,   434,  1769,  2844,   282,    66,  2428,   334,   215,   184,
             1,  1528,   659,  1794,     9,   111,  1860,  1752,   694,    66,
          4788,    18,    1

In [35]:
test_ds[0]

(tensor([[51, 52, 43,  ...,  0,  0,  0],
         [29,  0,  0,  ...,  0,  0,  0],
         [19, 25, 58,  ...,  0,  0,  0],
         ...,
         [52, 45, 33,  ...,  0,  0,  0],
         [32, 25, 10,  ...,  0,  0,  0],
         [49,  0,  0,  ...,  0,  0,  0]]),
 tensor([ 1515,   570,  1433,  1728,  4892,  2013,    67,   309,   215,  3156,
          3138,     9,     1,     1,     1,    67,   720,   950,     1,     1,
          1433,  1160,    14,  2639,   150,   184,  4391,  1526,  1708,    22,
            73,  4892,  1519,  2013,   659,   588,   215,    73,   353,  1827,
          1738,  1528,    18,   968,     9,   126,   309,  3683,   184,  3165,
             1,   960,   215,    14,   452,  1528,   150,    14,   353,    67,
          5514,     5,    73,  3156,  2028,  3138,     5,     1,     1,     9,
           309,   983,   831,   150,    14,  1528,    80,  3683,  3072,  3241,
          4843,    25,    14,  2074,  2010,   977,     1,  1885,     1,     1,
           678,  4932,   15

In [36]:
# assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
# assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
# assert len(test_dataset) == 3683, "Неправильная длина test_dataset"
#
# assert torch.equal(train_dataset[0][0], torch.tensor([2,1,3,4,5,6,7,8,9])), "Неправильно сформированный train_dataset"
# assert torch.equal(train_dataset[0][1], torch.tensor([3,0,2,0,0,0,2,0,0])), "Неправильно сформированный train_dataset"
#
# assert torch.equal(valid_dataset[0][0], torch.tensor([1737,571,1777,197,687,145,349,111,1819,1558,9])), "Неправильно сформированный valid_dataset"
# assert torch.equal(valid_dataset[0][1], torch.tensor([0,0,3,0,0,0,0,0,0,0,0])), "Неправильно сформированный valid_dataset"
#
# assert torch.equal(test_dataset[0][0], torch.tensor([1516,571,1434,1729,4893,2014,67,310,215,3157,3139,9])), "Неправильно сформированный test_dataset"
# assert torch.equal(test_dataset[0][1], torch.tensor([0,0,1,0,0,0,0,4,0,0,0,0])), "Неправильно сформированный test_dataset"
#
# print("Тесты пройдены!")



print('Я по-другому формирую датасеты, поэтому эти тесты не пройдутся')

Я по-другому формирую датасеты, поэтому эти тесты не пройдутся


Для того, чтобы дополнять последовательности паддингом, будем использовать параметр `collate_fn` класса `DataLoader`.

Принимая последовательность пар тензоров для предложений и тегов, необходимо дополнить все последовательности до последовательности максимальной длины в батче.

Используйте для дополнения спецтокен `<PAD>` для последовательностей слов / токенов и -1 для последовательностей тегов.

**hint**: удобно использовать метод **torch.nn.utils.rnn**. Обратите особое внимание на параметр *batch_first*

`Collator` можно реализовать двумя способами:
- класс с методом `__call__`
- функцию

Мы пойдем первым путем.

Инициализировать экземпляр класса `Collator` (метод `__init__`) с помощью двух параметров:
- id `<PAD>` спецтокена для последовательностей слов / токенов
- id `<PAD>` спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов
    
P.S. `<PAD>` значение нужно для того, чтобы при подсчете лосса легко отличать западдированные токены от других. Можно использовать параметр *ignore_index* при инициализации лосса.

**Задание. Реализуйте класс коллатора NERCollator.** **<font color='red'>(1 балл)</font>**

In [37]:
from torch.nn.utils.rnn import pad_sequence


class NERCollator:
    """
    Collator that handles variable-size sentences.
    """

    def __init__(self, char_padding_value: int, token_padding_value: int, label_padding_value: int):
        self.char_padding_value = char_padding_value
        self.token_padding_value = token_padding_value
        self.label_padding_value = label_padding_value

    def __call__(self, batch: List[Tuple[LongTensor, ...]]) -> Dict[str, LongTensor]:
        chars, tokens, labels = zip(*batch)
        return {
            "char_ids": pad_sequence(chars, batch_first=True, padding_value=self.char_padding_value).long(),
            "token_ids": pad_sequence(tokens, batch_first=True, padding_value=self.token_padding_value).long(),
            "labels": pad_sequence(labels, batch_first=True, padding_value=self.label_padding_value).long()
        }

In [38]:
coll = NERCollator(
    char_padding_value=c2idx['<PAD>'],
    token_padding_value=t2idx["<PAD>"],
    label_padding_value=-1,
)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [39]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=coll)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=128, shuffle=False, collate_fn=coll)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1,shuffle=False, collate_fn=coll)

Посмотрим на то, что мы получили:

In [40]:
batch = next(iter(train_dl))

chars = batch['char_ids'].to(DEVICE)
tokens = batch['token_ids'].to(DEVICE)
labels = batch['labels'].to(DEVICE)

In [41]:
chars, chars.shape

(tensor([[[51, 52, 43,  ...,  0,  0,  0],
          [29,  0,  0,  ...,  0,  0,  0],
          [13, 52, 45,  ...,  0,  0,  0],
          ...,
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0]],
 
         [[ 8, 33, 45,  ...,  0,  0,  0],
          [29,  0,  0,  ...,  0,  0,  0],
          [55, 33, 51,  ...,  0,  0,  0],
          ...,
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0]],
 
         [[51, 52, 43,  ...,  0,  0,  0],
          [29,  0,  0,  ...,  0,  0,  0],
          [33,  6, 55,  ...,  0,  0,  0],
          ...,
          [47,  0,  0,  ...,  0,  0,  0],
          [26, 29,  2,  ...,  0,  0,  0],
          [42,  0,  0,  ...,  0,  0,  0]],
 
         ...,
 
         [[43, 37, 43,  ...,  0,  0,  0],
          [29,  0,  0,  ...,  0,  0,  0],
          [44, 52, 55,  ...,  0,  0,  0],
          ...,
          [ 0,  0,  0,  ...,  0, 

In [42]:
tokens, tokens.shape

(tensor([[1515,  570, 8464,  ...,    0,    0,    0],
         [1635,  570, 1148,  ...,    0,    0,    0],
         [1515,  570,   15,  ...,  122, 2084,  124],
         ...,
         [7803,  570, 1410,  ...,    0,    0,    0],
         [5754, 3418,    1,  ...,    0,    0,    0],
         [4923,    1,  963,  ...,    0,    0,    0]], device='cuda:0'),
 torch.Size([8, 506]))

In [43]:
labels, labels.shape

(tensor([[ 0,  0,  1,  ..., -1, -1, -1],
         [ 0,  0,  0,  ..., -1, -1, -1],
         [ 0,  0,  2,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ..., -1, -1, -1],
         [ 3,  0,  0,  ..., -1, -1, -1],
         [ 4,  8,  0,  ..., -1, -1, -1]], device='cuda:0'),
 torch.Size([8, 506]))

In [44]:
# train_tokens, train_labels = next(iter(
#     torch.utils.data.DataLoader(
#         train_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(train_tokens, torch.tensor([[ 2,  1,  3,  4,  5,  6,  7,  8,  9], [10, 11,  0,  0,  0,  0,  0,  0,  0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(train_labels, torch.tensor([[ 3,  0,  2,  0,  0,  0,  2,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"
#
# valid_tokens, valid_labels = next(iter(
#     torch.utils.data.DataLoader(
#         valid_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(valid_tokens, torch.tensor([[ 1737,   571,  1777,   197,   687,   145,   349,   111,  1819,  1558, 9], [  248, 10679,     0,     0,     0,     0,     0,     0,     0,     0,    0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(valid_labels, torch.tensor([[ 0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0], [ 1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"
#
# test_tokens, test_labels = next(iter(
#     torch.utils.data.DataLoader(
#         test_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(test_tokens, torch.tensor([[1516,  571, 1434, 1729, 4893, 2014,   67,  310,  215, 3157, 3139,    9], [   1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(test_labels, torch.tensor([[ 0,  0,  1,  0,  0,  0,  0,  4,  0,  0,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

print("Опять же, вид всех тензоров у меня другой, поэтому в этих тестах нет смысла")

Опять же, вид всех тензоров у меня другой, поэтому в этих тестах нет смысла


## Часть 2. BiLSTM-теггер (6 баллов)

Определите архитектуру сети, используя библиотеку PyTorch. 

Ваша архитектура в этом пункте должна соответствовать стандартному теггеру:
* Embedding слой на входе
* LSTM (однонаправленный или двунаправленный)слой для обработки последовательности
* Dropout (заданный отдельно или встроенный в LSTM) для уменьшения переобучения
* Linear слой на выходе

Для обучения сети используйте поэлементную кросс-энтропийную функцию потерь.

**Обратите внимание**, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь. В качестве оптимизатора рекомендуется использовать Adam. Для получения значений предсказаний по выходам модели используйте функцию `argmax`.

**Задание. Реализуйте класс модели BiLSTM.** **<font color='red'>(2 балл)</font>**

In [93]:
from typing import Union
from torch import Tensor, BoolTensor
from torch.nn.init import xavier_uniform_
from torch.nn import Embedding, LSTM, Linear, Dropout, Sequential, Conv1d, Module, ReLU, LayerNorm, Parameter, \
    MaxPool1d, Conv2d, CrossEntropyLoss

from torchcrf import CRF


class Transpose(Module):

    def __init__(self, *dims):
        super().__init__()
        self._dims = dims

    def forward(self, x):
        return torch.transpose(x, *self._dims)


def view_input_tensors_factory(*positional_dims, **keyword_dims):
    def view_input_tensors(fn):
        def wrapper(*args, **kwargs):
            return fn(
                *(x.view(*dims) for x, dims in zip(args, positional_dims)),
                **{name: x.view(*keyword_dims[name]) for name, x in kwargs.items()}
            )
        return wrapper
    return view_input_tensors


def add_kwarg_factory(name: str, value: Any):
    def add_kwarg(fn):
        def wrapper(*args, **kwargs):
            return fn(*args, **kwargs, **{name: value})
        return wrapper
    return add_kwarg


def add_mask_to_kwargs_factory(mask_value: Any, build_by: Union[str, int], mask_name: str):
    def add_mask_to_kwargs(fn):
        def wrapper(*args, **kwargs):
            if isinstance(build_by, int):
                target = args[build_by]
            else:
                target = kwargs[build_by]
            mask = (target != mask_value)
            return fn(*args, **kwargs, **{mask_name: mask})
        return wrapper
    return add_mask_to_kwargs


def convert_output_to_tensor_factory(dtype=None, device=None):
    def convert_output_to_tensor(fn):
        def wrapper(*args, **kwargs) -> Tensor:
            return torch.tensor(fn(*args, **kwargs), dtype=dtype, device=device)
        return wrapper
    return convert_output_to_tensor


@dataclass
class AttentionParameters:
    dropout: float
    dims: int
    num_heads: int


class SelfAttention(Module):

    def __init__(self, attention: Module):
        super().__init__()
        self._attention = attention

    def forward(self, x):
        res, _ = self._attention(x, x, x)
        return res


class ResidualConnection(Module):

    def __init__(self, module: Module):
        super().__init__()
        self._module = module

    def forward(self, x):
        wrapped_res = self._module(x)
        return x + wrapped_res


class BiLSTM(Module):
    """
    Bidirectional LSTM architecture.
    """

    def __init__(
            self,
            num_chars: int,
            vocab: Dict[str, int],
            char_embedding_dim: int,
            pretrained_embeddings: Optional[Path],
            freeze_pretrained_embeddings: bool,
            max_token_length: int,
            token_embedding_dim: int,
            hidden_size: int,
            num_layers: int,
            dropout: float,
            interlayer_dropout: float,
            bidirectional: bool,
            n_classes: int,
            char_padding_idx: int,
            token_padding_idx: int,
            head_type: str,
            criterion_type: str,
            train_initial_state: bool,
            attention_params: Optional[AttentionParameters] = None
    ):
        super().__init__()

        self._char_embed = Embedding(num_chars, char_embedding_dim, padding_idx=char_padding_idx)
        self._char_conv = Conv2d(1, char_embedding_dim, kernel_size=(3, char_embedding_dim), padding=(1, 0))
        self._char_pool = MaxPool1d(kernel_size=max_token_length)
        self._char_norm = LayerNorm(char_embedding_dim)
        self._char_dropout = Dropout(dropout)

        self._padding_idx = token_padding_idx

        if pretrained_embeddings is None:
            embeddings = {}
        else:
            embeddings = read_glove(pretrained_embeddings)

        self._pretrained_embedding = Embedding(len(embeddings), token_embedding_dim)
        self._pretrained_threshold = len(embeddings)

        for word in embeddings.keys():
            word_idx = vocab[word]
            if word_idx >= self._pretrained_threshold:
                # the first N words in vocabulary are expected to be pretrained
                raise ValueError(f'Malformed vocabulary! Pretrained word {word} has {word_idx} idx while pretrained threshold is {self._pretrained_threshold}!')

            word_embedding = embeddings[word]
            with torch.no_grad():
                self._pretrained_embedding.weight[word_idx] = torch.tensor(word_embedding)

        if token_padding_idx < self._pretrained_threshold:
            raise ValueError('Padding idx is expected to be pretrained!')

        self._true_padding_idx = token_padding_idx - self._pretrained_threshold
        self._untrained_embedding = Embedding(len(vocab) - len(embeddings), token_embedding_dim, padding_idx=self._true_padding_idx)

        if freeze_pretrained_embeddings:
            self._pretrained_embedding.weight.requires_grad = False

        embedding_dim = token_embedding_dim + char_embedding_dim
        self._embed_dropout = Dropout(dropout)

        self._rnn = LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=interlayer_dropout,
            bidirectional=bidirectional
        )
        bi = (2 if bidirectional else 1)
        self._train_initial_state = train_initial_state
        self._h0 = Parameter(torch.empty(bi * num_layers, 1, hidden_size))
        self._c0 = Parameter(torch.empty(bi * num_layers, 1, hidden_size))
        xavier_uniform_(self._h0.data)
        xavier_uniform_(self._c0.data)

        lstm_dims = hidden_size * bi

        if head_type == 'linear':
            self._head = Sequential(
                Dropout(dropout),
                Linear(lstm_dims, n_classes)
            )
        elif head_type == 'conv':
            self._head = Sequential(
                LayerNorm(lstm_dims),
                Dropout(dropout),
                Transpose(-2, -1),
                Conv1d(lstm_dims, n_classes, kernel_size=3, padding=1),
                Transpose(-2, -1)
            )
        elif head_type == 'ff':
            self._head = Sequential(
                LayerNorm(lstm_dims),
                Dropout(dropout),
                Linear(lstm_dims, lstm_dims // 4),
                LayerNorm(lstm_dims // 4),
                ReLU(inplace=True),
                Dropout(dropout),
                Linear(lstm_dims // 4, lstm_dims // 16),
                LayerNorm(lstm_dims // 16),
                ReLU(inplace=True),
                Dropout(dropout),
                Linear(lstm_dims // 16, n_classes)
            )
        elif head_type == 'attention':
            if attention_params is None:
                raise ValueError

            self._head = Sequential(  # similar to transformer encoder block
                Dropout(dropout),
                Linear(lstm_dims, attention_params.dims),
                ReLU(inplace=True),
                LayerNorm(attention_params.dims),
                ResidualConnection(SelfAttention(torch.nn.MultiheadAttention(
                    embed_dim=attention_params.dims,
                    num_heads=attention_params.num_heads,
                    dropout=attention_params.dropout,
                    batch_first=True
                ))),
                Linear(attention_params.dims, n_classes)
            )
        else:
            raise ValueError

        if criterion_type == 'celoss':
            self._celoss = CrossEntropyLoss(ignore_index=-1)
            self._criterion = view_input_tensors_factory((-1, n_classes), (-1,))(self._celoss)
            self._decoder: Callable[[Tensor], LongTensor] = partial(torch.argmax, dim=-1)
            self._loss_postprocess = lambda x: x
        elif criterion_type == 'crf':
            self._crf = CRF(n_classes, batch_first=True)
            self._criterion = add_kwarg_factory(name='reduction', value='token_mean')(add_mask_to_kwargs_factory(mask_value=-1, build_by=1, mask_name='mask')(self._crf))
            self._decoder = convert_output_to_tensor_factory(dtype=torch.long, device=DEVICE)(self._crf.decode)
            self._loss_postprocess = lambda x: -x  # crf returns log-likelihood
        else:
            raise ValueError

    def _embed_tokens(self, token_ids: LongTensor) -> Tensor:
        token_repr = self._untrained_embedding(torch.full_like(token_ids, fill_value=self._true_padding_idx))
        pretrained_mask = (token_ids < self._pretrained_threshold)
        untrained_mask = ~pretrained_mask
        token_repr[pretrained_mask] = self._pretrained_embedding(token_ids[pretrained_mask])
        token_repr[untrained_mask] = self._untrained_embedding(token_ids[untrained_mask] - self._pretrained_threshold)

        return token_repr


    def _embed_chars(self, char_ids: LongTensor) -> Tensor:
        char_repr = self._char_embed(char_ids)  # (B, SL, TL, CH)
        char_repr = self._char_dropout(self._char_norm(char_repr))
        batch_size, sequence_length, token_length, char_embed_dims = char_repr.shape
        char_repr = self._char_conv(char_repr.view(-1, 1, token_length, char_embed_dims)).squeeze(-1)  # (B * SL, CH, TL)
        char_repr = self._char_pool(char_repr).squeeze(-1).view(batch_size, sequence_length, char_embed_dims)

        return char_repr


    def _lstm_pass(self, features: Tensor, padding_mask: BoolTensor) -> Tensor:
        # используем специальную функцию pack_padded_sequence для того, чтобы получить структуру PackedSequence
        # которая не учитывать паддинг при проходе rnn
        length = padding_mask.sum(dim=1).detach().cpu()
        packed_embed = torch.nn.utils.rnn.pack_padded_sequence(features, length, batch_first=True, enforce_sorted=False)

        # используем специальную функцию pad_packed_sequence для того, чтобы получить тензор из PackedSequence
        if not self._train_initial_state:
            packed_rnn_output, _ = self._rnn(packed_embed)
        else:
            batch_size = features.shape[0]
            packed_rnn_output, _ = self._rnn(packed_embed, (self._h0.repeat(1, batch_size, 1), self._c0.repeat(1, batch_size, 1)))
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_rnn_output, batch_first=True)
        return rnn_output


    def forward(self, char_ids: LongTensor, token_ids: LongTensor, labels: Optional[LongTensor] = None, return_predictions: bool = False, **_) -> Union[Tensor, Tuple[Tensor, LongTensor]]:
        embed = self._embed_dropout(torch.concat([self._embed_tokens(token_ids), self._embed_chars(char_ids)], dim=-1))
        rnn_output = self._lstm_pass(embed, (token_ids != self._padding_idx))

        label_scores = self._head(rnn_output)

        if labels is not None:
            loss = self._loss_postprocess(self._criterion(label_scores, labels))
            if return_predictions:
                return loss, self._decoder(label_scores)
            return loss

        return self._decoder(label_scores)

In [46]:
torch.__version__

'1.13.0+cu117'

In [47]:
m = BiLSTM(
    num_chars=len(c2idx),
    vocab=t2idx,
    char_embedding_dim=50,
    token_embedding_dim=100,
    pretrained_embeddings=None,
    hidden_size=100,
    num_layers=1,
    dropout=0.0,
    interlayer_dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    char_padding_idx=c2idx['<PAD>'],
    token_padding_idx=t2idx['<PAD>'],
    max_token_length=20,
    head_type='ff',
    criterion_type='crf',
    train_initial_state=True,
    freeze_pretrained_embeddings=True
).to(DEVICE)

In [48]:
m

BiLSTM(
  (_char_embed): Embedding(60, 50, padding_idx=0)
  (_char_conv): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1), padding=(1, 0))
  (_char_pool): MaxPool1d(kernel_size=20, stride=20, padding=0, dilation=1, ceil_mode=False)
  (_char_norm): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
  (_char_dropout): Dropout(p=0.0, inplace=False)
  (_pretrained_embedding): Embedding(0, 100)
  (_untrained_embedding): Embedding(10951, 100, padding_idx=0)
  (_embed_dropout): Dropout(p=0.0, inplace=False)
  (_rnn): LSTM(150, 100, batch_first=True, bidirectional=True)
  (_head): Sequential(
    (0): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (1): Dropout(p=0.0, inplace=False)
    (2): Linear(in_features=200, out_features=50, bias=True)
    (3): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=50, out_features=12, bias=True)
    (7): LayerNorm((12,), eps=1e-05, elementwise_affi

In [49]:
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [50]:
loss, pred = m(chars, tokens, labels, return_predictions=True)
pred.shape, loss

/home/viktor-sch/dl-2022/venv/lib/python3.8/site-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


(torch.Size([8, 506]), tensor(2.7243, device='cuda:0', grad_fn=<NegBackward0>))

In [51]:
assert pred.shape == torch.Size([8, 506])
assert 2 < loss < 3

print("Тесты пройдены!")

Тесты пройдены!


### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше 0.76. 

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [52]:
# создадим SummaryWriter для эксперимента с BiLSTMModel

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/BiLSTMModel")

**Задание. Реализуйте функцию подсчета метрик compute_metrics.** **<font color='red'>(1 балл)</font>**

In [53]:
from typing import Callable
from torch import Tensor


@dataclass
class Entity:
    start: int
    end: int
    type: str

    def __hash__(self):
        return hash((self.start, self.end, self.type))


def ent_type(lbl: str) -> str:
    return lbl.split('-')[1]


def is_ent(lbl: str) -> bool:
    return lbl.split('-')[0] in {'B', 'I'}


def is_ent_start(lbl: str) -> bool:
    return lbl.startswith('B')


# TODO: rewrite only for entity labels + their positions
def decode_labels(labels: Iterable[str]) -> Iterable[Entity]:

    prev_label = 'O'
    start: Optional[int] = None

    curr_idx = 0
    for curr_idx, label in enumerate(labels):
        if is_ent(label):
            if is_ent(prev_label):
                # ... X1-ENT1 X2-ENT2 ...
                if ent_type(label) != ent_type(prev_label) or is_ent_start(label):
                    # ENT1 != ENT2 or X2 == B - entity ended + start new entity
                    yield Entity(start, curr_idx, ent_type(prev_label))
                    start = curr_idx
                # X2 != B and ENT1 == ENT2 - do nothing
            else:
                # ... O X-ENT ... - start new entity
                start = curr_idx
        else:
            if is_ent(prev_label):
                # ... X-ENT O ... - entity ended
                yield Entity(start, curr_idx, ent_type(prev_label))
                start = None  # do not start new entity
            # ... O O ... - do nothing

        prev_label = label

    if is_ent(prev_label):
        # sequence ended on entity
        yield Entity(start, curr_idx + 1, ent_type(prev_label))


def safe(f: Callable[[], Any], default: Any = None) -> Any:
    try:
        return f()
    except:
        return default


def compute_metrics(preds: LongTensor, label_ids: LongTensor, label_mapping: np.ndarray, ignore_index: int = -1) -> Dict[str, float]:
    """
    Compute NER metrics. All input tensors should be ravelled.
    """

    metrics = {}
    label_mask = (label_ids != ignore_index)
    predicted_label_ids = preds[label_mask].detach().cpu().numpy()
    gold_label_ids = label_ids[label_mask].detach().cpu().numpy()

    predicted_labels = label_mapping[predicted_label_ids]
    gold_labels = label_mapping[gold_label_ids]

    predicted_entities = set(decode_labels(predicted_labels))
    gold_entities = set(decode_labels(gold_labels))

    true_positive = predicted_entities.intersection(gold_entities)
    false_positive = predicted_entities.difference(gold_entities)
    false_negative = gold_entities.difference(predicted_entities)

    recall_micro = safe(lambda: len(true_positive) / (len(true_positive) + len(false_negative)), 0.0)
    precision_micro = safe(lambda: len(true_positive) / (len(true_positive) + len(false_positive)), 0.0)
    f1_micro = safe(lambda: 2 * precision_micro * recall_micro / (precision_micro + recall_micro), 0.0)

    all_f1 = []
    all_precision = []
    all_recall = []
    all_weights = []

    for label in label_mapping:
        if not is_ent_start(label):
            continue

        target_type = ent_type(label)
        predicate = lambda entity: entity.type == target_type

        target_predicted_entities = set(filter(predicate, predicted_entities))
        target_gold_entities = set(filter(predicate, gold_entities))

        true_positive = target_predicted_entities.intersection(target_gold_entities)
        false_positive = target_predicted_entities.difference(target_gold_entities)
        false_negative = target_gold_entities.difference(predicted_entities)

        recall = safe(lambda: len(true_positive) / (len(true_positive) + len(false_negative)), 0.0)
        precision = safe(lambda: len(true_positive) / (len(true_positive) + len(false_positive)), 0.0)
        f1 = safe(lambda: 2 * precision * recall / (precision + recall), 0.0)

        metrics[f'recall_{target_type}'] = recall
        metrics[f'precision_{target_type}'] = precision
        metrics[f'f1_{target_type}'] = f1

        all_f1.append(f1)
        all_precision.append(precision)
        all_recall.append(recall)
        all_weights.append(len(target_gold_entities) / len(gold_entities))

    all_f1 = np.array(all_f1)
    all_recall = np.array(all_recall)
    all_precision = np.array(all_precision)

    recall_macro = all_recall.mean()
    precision_macro = all_precision.mean()
    f1_macro = all_f1.mean()

    recall_weighted = (all_recall * all_weights).sum()
    precision_weighted = (all_precision * all_weights).sum()
    f1_weighted = (all_f1 * all_weights).sum()

    metrics["precision_micro"] = precision_micro
    metrics["precision_macro"] = precision_macro
    metrics["precision_weighted"] = precision_weighted

    metrics["recall_micro"] = recall_micro
    metrics["recall_macro"] = recall_macro
    metrics["recall_weighted"] = recall_weighted

    metrics["f1_micro"] = f1_micro
    metrics["f1_macro"] = f1_macro
    metrics["f1_weighted"] = f1_weighted

    return metrics

**Задание. Реализуйте функции обучения и тестирования train_epoch и evaluate_epoch.** **<font color='red'>(2 балла)</font>**

In [54]:
from transformers import get_constant_schedule_with_warmup, get_linear_schedule_with_warmup, \
    get_cosine_schedule_with_warmup
from torch.cuda.amp import GradScaler
from copy import deepcopy
from torch.optim import Optimizer
from torch.nn import Module

from torch.utils.data import DataLoader


def validate(model: Module, dataloader: DataLoader, label_mapping: np.ndarray, ignore_index: int) -> Dict[str, float]:
    model.eval()

    # collect all revelled predictions

    all_predictions: List[Tensor] = []
    all_label_ids: List[LongTensor] = []
    all_losses: List[float] = []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            label_ids = batch['labels']

            loss, predictions = model(**batch, return_predictions=True)

            # ravel batch and length dims

            ravelled_predictions = predictions.detach().cpu().reshape(-1)
            ravelled_label_ids = label_ids.detach().cpu().reshape(-1)

            all_predictions.append(ravelled_predictions)
            all_label_ids.append(ravelled_label_ids)
            all_losses.append(loss.item())

        combined_predictions = torch.concat(all_predictions)
        combined_label_ids = torch.concat(all_label_ids)

    mean_loss = sum(all_losses) / len(all_losses)
    return {**compute_metrics(combined_predictions, combined_label_ids, label_mapping, ignore_index), 'loss': mean_loss}


def log_metrics(metrics: Dict[str, float], writer: SummaryWriter, *, step: int) -> None:
    for metric_name, metric_value in metrics.items():
        writer.add_scalar(metric_name, metric_value, global_step=step)


def get_state(model: Module) -> dict:
    model.cpu()
    state = deepcopy(model.state_dict())
    model.to(DEVICE)
    return state


def set_state(model: Module, state: dict) -> Module:
    model.cpu()
    model.load_state_dict(state)
    model.to(DEVICE)
    return model


def set_lr(optimizer: Optimizer, lr: float) -> None:
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


class DummyScheduler:

    def step(self, *args, **kwargs):
        pass


def train_loop(
        model: Module,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        label_mapping: np.ndarray,
        optimizer: Optimizer,
        batch_size: int,
        lr_schedule: str,
        initial_lr: float,
        drop_factor_lr: float,
        min_lr: float,
        max_epochs: int,
        reload_patience: int,
        clip_grads: bool,
        max_grad_norm: float,
        warmup_epochs: float,
        val_period: int,
        disable_amp: bool,
        target_metric: str,
        ignore_index: int,
        writer: SummaryWriter
) -> Module:
    """
    One training cycle (loop).
    """

    model.train()

    epoch = 0
    epoch_examples = len(train_dataloader) * batch_size
    examples_seen = 0

    first_time = True
    examples_from_last_log = 0
    logging_loss_sum = 0

    curr_lr = 0 if warmup_epochs > 0.0 else initial_lr
    curr_patience = reload_patience

    if lr_schedule == 'constant':
        scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=int(epoch_examples * warmup_epochs / batch_size))
    elif lr_schedule == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(epoch_examples * warmup_epochs / batch_size),
            num_training_steps=int(epoch_examples * max_epochs / batch_size)
        )
    elif lr_schedule == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(epoch_examples * warmup_epochs / batch_size),
            num_training_steps=int(epoch_examples * max_epochs / batch_size)
        )
    elif lr_schedule == 'adaptive':
        scheduler = DummyScheduler()
    else:
        raise ValueError

    best_model_state = get_state(model)
    best_metric = np.nan

    scaler = GradScaler()

    while (examples_seen <= epoch_examples * warmup_epochs or curr_lr > min_lr) and not epoch > max_epochs:
        epoch += 1
        print(f'{epoch} epoch, best metric: {best_metric:.3f}')
        for batch in train_dataloader:
            model.train()

            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            label_ids = batch['labels']

            if disable_amp:
                optimizer.zero_grad()
                loss = model(**batch)
                loss.backward()
                if clip_grads:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
            else:
                optimizer.zero_grad()
                with torch.cuda.amp.autocast():
                    loss = model(**batch)

                scaler.scale(loss).backward()
                if clip_grads:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                scaler.step(optimizer)
                scaler.update()

            scheduler.step()
            curr_lr = optimizer.param_groups[0]['lr']

            examples_seen += len(label_ids)

            logging_loss_sum += loss.item() * len(label_ids)
            examples_from_last_log += len(label_ids)

            if examples_from_last_log >= val_period:
                metrics = validate(model, val_dataloader, label_mapping, ignore_index)

                if examples_seen >= epoch_examples * warmup_epochs:
                    if first_time:
                        curr_lr = initial_lr
                        first_time = False
                        set_lr(optimizer, curr_lr)

                    target_metric_value = metrics[target_metric]
                    if best_metric is np.nan or target_metric_value > best_metric:
                        best_model_state = get_state(model)
                        best_metric = target_metric_value
                        curr_patience = reload_patience
                    else:
                        curr_patience -= 1
                        if curr_patience <= 0:
                            set_state(model, best_model_state)
                            curr_patience = reload_patience

                            if lr_schedule == 'adaptive':
                                curr_lr *= drop_factor_lr
                                set_lr(optimizer, curr_lr)
                else:
                    if lr_schedule == 'adaptive':
                        # warmup period
                        curr_lr = initial_lr * examples_seen / (epoch_examples * warmup_epochs)
                        set_lr(optimizer, curr_lr)

                metrics['train_loss'] = logging_loss_sum / examples_from_last_log
                examples_from_last_log = 0
                logging_loss_sum = 0

                metrics['learning_rate'] = curr_lr
                metrics['epoch'] = examples_seen / epoch_examples

                log_metrics(metrics, writer, step=examples_seen)

    set_state(model, best_model_state)
    return model

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**


In [52]:
!rm -rd logs

In [55]:
label_mapping = {v: k for k, v in label2idx.items()}
label_mapping = np.array([label_mapping[idx] for idx in range(len(label_mapping))])

exp_id = 0

In [61]:
embeddings_path = Path('glove/data/glove.6B.300d.txt')

convert_to_iob = True
train_docs = read_conll2003(Path('conll03/data/train.txt'), convert_to_iob=convert_to_iob, lower=False)
val_docs = read_conll2003(Path('conll03/data/valid.txt'), convert_to_iob=convert_to_iob, lower=False)
test_docs = read_conll2003(Path('conll03/data/test.txt'), convert_to_iob=convert_to_iob, lower=False)

lower = True
char2idx = get_char2idx(token2cnt.keys(), num_embedding=False)
token2cnt = get_token_counts(train_docs, lower=lower)
token2idx = get_token2idx(token2cnt, min_count=10, base_dict=glove2vocab(embeddings_path))

collator = NERCollator(
    char_padding_value=c2idx['<PAD>'],
    token_padding_value=t2idx['<PAD>'],
    label_padding_value=-1,
)

max_token_length = 20
max_sequence_length = 100000000000000000000

train_dataset = NERDataset(
    train_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)
valid_dataset = NERDataset(
    val_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)
test_dataset = NERDataset(
    test_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)

train_batch_size = 32
eval_batch_size = 512

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collator)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=eval_batch_size, shuffle=False, collate_fn=collator)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=eval_batch_size,shuffle=False, collate_fn=collator)

Read 946 documents and 14041 sentences from conll03/data/train.txt
Read 216 documents and 3250 sentences from conll03/data/valid.txt
Read 231 documents and 3453 sentences from conll03/data/test.txt
Building pretrained vocabulary from glove/data/glove.6B.300d.txt
Vocabulary size: 400000
Final vocabulary size: 400023


In [62]:
from pprint import pprint

exp_id += 1

writer = SummaryWriter(log_dir=f"logs/BiLSTMModel_{exp_id}")

lr = 0.015
model = BiLSTM(
    num_chars=len(char2idx),
    vocab=token2idx,
    char_embedding_dim=50,
    token_embedding_dim=300,
    pretrained_embeddings=embeddings_path,
    freeze_pretrained_embeddings=True,
    hidden_size=500,
    num_layers=1,
    dropout=0.3,
    interlayer_dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    max_token_length=max_token_length,
    char_padding_idx=char2idx['<PAD>'],
    token_padding_idx=token2idx['<PAD>'],
    head_type='linear',
    criterion_type='crf',
    train_initial_state=True
).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model = train_loop(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=valid_dataloader,
    label_mapping=label_mapping,
    optimizer=optimizer,
    batch_size=train_batch_size,
    initial_lr=lr,
    drop_factor_lr=0.5,
    min_lr=1e-4,
    reload_patience=10,
    lr_schedule='adaptive',
    max_epochs=2000,
    clip_grads=True,
    max_grad_norm=1.0,
    warmup_epochs=0.0,
    val_period=500,
    disable_amp=False,
    target_metric='f1_macro',
    ignore_index=-1,
    writer=writer
)

Extracting word embeddings from glove/data/glove.6B.300d.txt
Extracted 400000 embeddings
1 epoch, best metric: nan
2 epoch, best metric: 0.148
3 epoch, best metric: 0.749
4 epoch, best metric: 0.842
5 epoch, best metric: 0.865
6 epoch, best metric: 0.892
7 epoch, best metric: 0.900
8 epoch, best metric: 0.900
9 epoch, best metric: 0.903
10 epoch, best metric: 0.914
11 epoch, best metric: 0.916
12 epoch, best metric: 0.916
13 epoch, best metric: 0.916
14 epoch, best metric: 0.922
15 epoch, best metric: 0.922
16 epoch, best metric: 0.922
17 epoch, best metric: 0.922
18 epoch, best metric: 0.922
19 epoch, best metric: 0.923
20 epoch, best metric: 0.923
21 epoch, best metric: 0.926
22 epoch, best metric: 0.926
23 epoch, best metric: 0.926
24 epoch, best metric: 0.926
25 epoch, best metric: 0.926
26 epoch, best metric: 0.926
27 epoch, best metric: 0.926
28 epoch, best metric: 0.926
29 epoch, best metric: 0.926
30 epoch, best metric: 0.926
31 epoch, best metric: 0.927
32 epoch, best metric: 

In [63]:
metrics = validate(model, valid_dataloader, label_mapping, -1)
pprint(metrics)

{'f1_LOC': 0.9653856636685746,
 'f1_MISC': 0.8841870824053452,
 'f1_ORG': 0.903731343283582,
 'f1_PER': 0.9636264929424538,
 'f1_macro': 0.9292326455749889,
 'f1_micro': 0.9385408741229182,
 'f1_weighted': 0.9383252026780184,
 'loss': 0.04128353297710419,
 'precision_LOC': 0.9630233822729745,
 'precision_MISC': 0.9002267573696145,
 'precision_ORG': 0.9044062733383121,
 'precision_PER': 0.9584233261339092,
 'precision_macro': 0.9315199347787027,
 'precision_micro': 0.9389377537212449,
 'precision_weighted': 0.938614228801651,
 'recall_LOC': 0.9677595628415301,
 'recall_MISC': 0.8687089715536105,
 'recall_ORG': 0.9030574198359433,
 'recall_PER': 0.9688864628820961,
 'recall_macro': 0.927103104278295,
 'recall_micro': 0.9381443298969072,
 'recall_weighted': 0.9381443298969072}


In [64]:
metrics = validate(model, test_dataloader, label_mapping, -1)
pprint(metrics)

{'f1_LOC': 0.9264926492649265,
 'f1_MISC': 0.7897435897435897,
 'f1_ORG': 0.8688029020556227,
 'f1_PER': 0.9534520462355514,
 'f1_macro': 0.8846227968249226,
 'f1_micro': 0.9005086106897475,
 'f1_weighted': 0.9001950797436944,
 'loss': 0.09486447274684906,
 'precision_LOC': 0.9262147570485902,
 'precision_MISC': 0.8117469879518072,
 'precision_ORG': 0.8651414810355208,
 'precision_PER': 0.9543464665415885,
 'precision_macro': 0.8893624231443767,
 'precision_micro': 0.9025219102128421,
 'precision_weighted': 0.9020405061364487,
 'recall_LOC': 0.9267707082833133,
 'recall_MISC': 0.7689015691868759,
 'recall_ORG': 0.8724954462659381,
 'recall_PER': 0.9525593008739076,
 'recall_macro': 0.8801817561525087,
 'recall_micro': 0.8985042735042735,
 'recall_weighted': 0.8985042735042735}


Вообще с этой архитектурой можно выбить 0.9 на тесте, но мне надоело.

Влияние контекста (f1_macro на тесте):

| 1 пример = 1 предложение | 64     | 128    | 256    | 512    | 1 пример = 1 документ |
|--------------------------|--------|--------|--------|--------|-----------------------|
| 0.8709                   | 0.8726 | 0.8811 | 0.8815 | 0.8857 | 0.8846                |

## Часть 3. Transformers-теггер (6 баллов)

В данной части задания нужно сделать все то же самое, но с использованием модели на базе архитектуры Transformer, а именно предлагается дообучать предобученную модель **BERT**.

Для данной модели подразумевается специальная подготовка данных, с чего мы и начнем:

Модель **BERT** использует специальный токенизатор WordPiece для разбиения предложений на токены. Готовая предобученная версия такого токенизатора существует в библиотеке **transformers**. Есть два класса: `BertTokenizer` и `BertTokenizerFast`. Использовать можно любой, но второй вариант работает существенно быстрее.

Токенизаторы можно обучать с нуля на своем корпусе данных, а можно подгружать уже готовые. Готовые токенизаторы, как правило, соответствуют предобученной конфигурации модели, которая использует словарь из этого токенизатора. 

Мы будем использовать базовую конфигурацию предобученного **BERT** для модели и токенизатора.

P.S. Часто приходится проводить эксперименты с моделями разной архитектуры, например **BERT** и **GPT**, поэтому удобно использовать класс `AutoTokenizer`, который по названию модели сам определит, какой класс нужен для инициализации токенизатора.

In [56]:
from transformers import AutoTokenizer

In [57]:
model_name = "distilbert-base-cased"

Подгружение предобученных моделей и токенизаторов в **huggingface** происходит с помощью конструктора **from_pretrained**.

В данном конструкторе можно указать либо путь к предобученному токенизатору, либо название предобученной конфигурации, как в нашем случае: тогда **transformers** сам подгрузит нужные параметры:

In [58]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Подготовка словарей

В сравнении с рекуррентными моделями, на больше не нужно заниматься сборкой словаря, так как это уже сделано заранее благодаря токенизаторам и алгоритмам, стоящими за ними.

Но нам как и прежде потребуется:
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0);

Но данное отображение у нас уже реализовано в одной из предыдущих частей задания.

### Подготовка датасета и загрузчика

Мы также хотим обучать модель батчами, поэтому нам как и прежде понадобятся `Dataset`, `Collator` и `DataLoader`.

Но мы не можем переиспользовать те, что в предыдущих частях задания, так как обработка данных должна производится немного иначе с использованием токенизатора.

Давайте напишем новый кастомный датасет, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов

и возвращать из метода `__getitem__` два списка:
- список текстовых значений (`List[str]`) из индексов токенов в сэмпле
- список целочисленных значений (`List[int]`) из индексов соответвующих тегов

P.S. В отличие от предыдущего кастомного датасет, здесь мы возвращаем два `List`'а вместо `torch.LongTensor`, так как логику формирования западдированного батча мы перенесем в `Collator` из-за специфики работы токенизатора - он сам возвращает уже западдированный тензор с индексами токенов, а для индексов тегов нам нужно будет сделать это самостоятельно по аналогии с предыдущим датасетом.

**Задание. Реализуйте класс датасета TransformersDataset.** **<font color='red'>(1 балл)</font>**

In [59]:
from transformers import PreTrainedTokenizer


def tokenize_document(doc: Document, label2idx: Dict[str, int], tokenizer: PreTrainedTokenizer) -> TokenizedDocument:

    def process_label(label: str, desired_length: int) -> List[str]:
        if is_ent_start(label):
            return [label] + [f'I-{ent_type(label)}'] * (desired_length - 1)
        return [label] * desired_length

    def tokenize_sentence(sentence: Iterable[str], labels: Iterable[str]) -> Tuple[LongTensor, LongTensor]:
        all_subtokens = []
        all_labels = []

        for token, label in zip(sentence, labels):
            subtokens = tokenizer.encode([token], add_special_tokens=False, is_split_into_words=True)
            if not len(subtokens):  # in case tokenizer fails to split token into sub tokens
                subtokens = [tokenizer.unk_token_id]
            all_subtokens.extend(subtokens)

            labels = map(label2idx.__getitem__, process_label(label, desired_length=len(subtokens)))
            all_labels.extend(labels)

        return torch.tensor(all_subtokens, dtype=torch.long).long(), torch.tensor(all_labels, dtype=torch.long).long()

    processed_sentences = []
    processed_labels = []
    fake_chars = []
    for sentence, labels in zip(doc.sentences, doc.labels):
        tokenized_sentence, tokenized_labels = tokenize_sentence(sentence, labels)
        processed_sentences.append(tokenized_sentence)
        processed_labels.append(tokenized_labels)
        fake_chars.append(torch.tensor([[-1]] * len(tokenized_sentence), dtype=torch.long))

    return TokenizedDocument(fake_chars, processed_sentences, processed_labels)  # we do not need character-level info

class TransformersDataset(torch.utils.data.Dataset):
    """
    Transformers Dataset for NER.
    """

    def __init__(
            self,
            docs: Iterable[Document],
            tokenizer: PreTrainedTokenizer,
            label2idx: Dict[str, int],
            max_sequence_length: int
    ):
        tokenized_documents = map(partial(tokenize_document, label2idx=label2idx, tokenizer=tokenizer), docs)
        sentence_grouper = partial(group_sentences, max_sequence_length=max_sequence_length)
        self._examples: List[Tuple[LongTensor, ...]] = list(chain.from_iterable(map(sentence_grouper, tokenized_documents)))

    def __len__(self):
        return len(self._examples)

    def __getitem__(self, idx: int) -> Tuple[LongTensor, ...]:
        return self._examples[idx][1:]  # skip char info


Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [60]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = TransformersDataset(train_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=512)
valid_dataset = TransformersDataset(val_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=512)
test_dataset = TransformersDataset(test_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=512)

Посмотрим на то, что мы получили:

In [61]:
train_dataset[0]

(tensor([  174,  1358, 22961,   176, 14170,  1840,  1106, 21423,  9304, 10721,
          1324,  2495, 12913,   119, 11109,  1200,  1602,  6715,  9304, 13356,
          5999,  1820,   118,  4775,   118,  1659,  1103, 27772,  3186,  1389,
          5500,  1163,  1113, 24438,  7719,  6194,  1122, 19786,  1114,   176,
         14170,  5566,  1106, 11060,  1106,   188, 17315,  9304, 10721,  1324,
          2495, 12913,  1235,  6479,  4959,  2480,  6340, 13991,  3653,  1169,
          1129, 12086,  1106,  8892,   119,   176, 14170,  1183,   112,   188,
          4702,  1106,  1103, 27772,  3186,  1389,  3779,   112,   188, 27431,
          3914,  1195, 18056,   195,  7635,  4119,  1163,  1113, 26055,  3965,
          6194, 11060,  1431,  4417,  8892,  3263,  2980,  1121,  2182,  1168,
          1190,  9304,  5168,  1394,  1235,  1103,  3812,  5566,  1108, 27830,
           119,   107,  1195,  1202,   183,   112,   189,  1619,  1251,  1216,
         13710,  1272,  1195,  1202,   183,   112,  

In [62]:
valid_dataset[0]

(tensor([ 5428,   118,  5837, 18117,  5759, 15189,  1321,  1166,  1120,  1499,
          1170,  6687,  2681,   119, 25338, 17996,  1820,   118,  4775,   118,
          1476,  1745,  1107, 10359,  1155,   118,  1668,  1200,   185, 20473,
         27466,  6262,  4199,  1261,  1300,  1111,  3383,  1113,   175, 22977,
          1183,  1112,  5837, 18117,  5759, 15189,  3222,  1199, 15955,  1204,
          1118,  1126,  6687,  1105,  3614,  2326,  1107,  1160,  1552,  1106,
          1321,  1166,  1120,  1103,  1246,  1104,  1103,  2514,  2899,   119,
          1147,  2215,  1113,  1499,   117,  1463,   117,  1336,  1129,  1603,
           118,  2077,  1112,  1641,  9521, 13936, 23152,   117, 25851,  6662,
          1105,  8910, 12210,  1155,  1804,  1107,  1113,  2681,  1229,   180,
          3452,  1189,  1146,  1111,  1575,  1159,  1107,  1147,  4458,   118,
          4634,  1801,  1222,  1136,  1916,  2522,  6662,   119,  1170, 11518,
          1199, 15955,  1204,  1149,  1111,  6032,  

In [63]:
test_dataset[0]

(tensor([ 5862,   118,   179, 26519,  1179,  1243,  6918,  1782,   117,  5144,
          1161,  1107,  3774,  3326,   119,  9468,  3309,  1306, 19122,  2293,
          2393,   118,  9562,   117, 10280,   170, 17952,  9712,  5132,  3052,
          1820,   118,  1367,   118,  5037,   179, 26519,  1179,  1310,  1103,
          6465,  1104,  1147,  1112,  1811,  4355,  1641,  1114,   170,  6918,
           123,   118,   122,  1782,  1222,   188, 12577,  1465,  1107,   170,
          1372,   172,  2899,  1801,  1113,   175, 22977,  1183,   119,  1133,
          5144,  1161,  1486,  1147,  6920,  6941,  1172,  1107,  1103,  1248,
          1801,  1104,  1103,  1372,   117, 13423,  1106,   170,  3774,   123,
           118,   121,  3326,  1106, 25551,  1116,   190,  1584, 17327, 20300,
           119,  5144,  1161,  4013,  1211,  1104,  1103,  1801,  1105,  1486,
          1317,  9820,  4007,  1235,  1103,  5603,  1582,  2517,  1165,   190,
          1584, 17327, 13074,   178, 18791,   188,  

In [64]:
# assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
# assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
# assert len(test_dataset) == 3683, "Неправильная длина test_dataset"
#
# assert train_dataset[0][0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Неправильно сформированный train_dataset"
# assert train_dataset[0][1] == [3,0,2,0,0,0,2,0,0], "Неправильно сформированный train_dataset"
#
# assert valid_dataset[0][0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Неправильно сформированный valid_dataset"
# assert valid_dataset[0][1] == [0,0,3,0,0,0,0,0,0,0,0], "Неправильно сформированный valid_dataset"
#
# assert test_dataset[0][0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Неправильно сформированный test_dataset"
# assert test_dataset[0][1] == [0,0,1,0,0,0,0,4,0,0,0,0], "Неправильно сформированный test_dataset"
#
# print("Тесты пройдены!")

print('Reject asserts imbrace monke')

Reject asserts imbrace monke


Реализуем новый `Collator`.

Инициализировать коллатор будет 3 аргументами:
- токенизатор
- параметры токенизатора в виде словаря (затем используем как `**kwargs`)
- id спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов

**Задание. Реализуйте класс коллатора TransformersCollator.** **<font color='red'>(2 балла)</font>**

In [65]:
class TransformersCollator:
    """
    Collator that handles variable-size sentences.
    """

    def __init__(self, token_padding_value: int, label_padding_value: int):
        self.token_padding_value = token_padding_value
        self.label_padding_value = label_padding_value

    def __call__(self, batch: List[Tuple[LongTensor, LongTensor]]) -> Dict[str, LongTensor]:
        tokens, labels = zip(*batch)
        masks = list(map(partial(torch.ones_like, dtype=torch.bool), tokens))
        return {
            "input_ids": pad_sequence(tokens, batch_first=True, padding_value=self.token_padding_value).long(),
            "labels": pad_sequence(labels, batch_first=True, padding_value=self.label_padding_value).long(),
            "attention_mask": pad_sequence(masks, batch_first=True, padding_value=False).long()
        }

In [66]:
collator = TransformersCollator(token_padding_value=tokenizer.pad_token_id, label_padding_value=-100)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [67]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

Посмотрим на то, что мы получили:

In [68]:
batch = next(iter(train_dataloader))

batch = {k: v.to(DEVICE) for k, v in batch.items()}

tokens = batch['input_ids']
labels = batch['labels']

In [69]:
tokens

tensor([[ 5428,   118,  4035,  1403,  1931,   191,   185,  8552, 13946,  1509,
          2774,  2794,  4015,   119, 25338, 17996,  1820,   118,  4775,   118,
          1572,  2794,  4015,  1113,  1103,  1503,  1285,  1104,  1103,  1503,
          1105,  1509,  2774,  1206,  4035,  1403,  1931,  1105,   185,  8552,
         13946,  1120,  1103, 13102,  1113,  2068,  2149,  6194,   131,  4035,
          1403,  1931,  1148,  6687,  2724,  1545,   113,   179,   119, 14869,
          2254,  9920,   117,   176,   119, 24438,  1766,  3186,  4335,   132,
         20049, 24750,  1197,  1128,  7221,   125,   118,  4573,   114,   185,
          8552, 13946,  1148,  6687,   113, 12292, 25325,   118,   122,   114,
         21718, 11394,  1126,  7200,   172,   172, 27421,   171,  1884,  4661,
         20039,   170, 11787,  1197,  1177, 10390,  1233,   172,  1884,  4661,
           171,   172, 27421,  3993,   178,  3174,  1584, 18257,  4611,   172,
         26036,  9349,   171,   182, 11781,  2716,  

In [70]:
labels

tensor([[   0,    0,    1,    5,    5,    0,    1,    5,    5,    0,    0,    0,
            0,    0,    1,    5,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    5,
            5,    0,    1,    5,    5,    0,    1,    5,    0,    0,    0,    0,
            0,    1,    5,    5,    0,    0,    0,    0,    0,    4,    8,    8,
            8,    0,    0,    4,    8,    8,    8,    8,    0,    0,    4,    8,
            8,    4,    8,    0,    0,    0,    0,    1,    5,    5,    0,    0,
            0,    0,    0,    0,    0,    0,    4,    8,    8,    8,    0,    4,
            8,    0,    4,    8,    0,    4,    8,    8,    8,    8,    8,    0,
            4,    8,    0,    4,    8,    0,    4,    8,    8,    8,    8,    0,
            4,    8,    0,    4,    8,    8,    0,    4,    8,    8,    8,    8,
            8,    8,    8,    0,    4,    8,    8,    0,    4,    8,    8,    0,
            4,    8,    8,  

In [71]:
# train_tokens, train_labels = next(iter(
#     torch.utils.data.DataLoader(
#         train_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(train_tokens['input_ids'], torch.tensor([[  101,   174,  1358, 22961,   176, 14170,  1840,  1106, 21423,  9304, 10721,  1324,  2495, 12913,   119,   102], [  101, 11109,  1200,  1602,  6715,   102,     0,     0,     0,     0,    0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(train_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(train_labels, torch.tensor([[-1,  3, -1,  0,  2, -1,  0,  0,  0,  2, -1, -1,  0, -1,  0, -1], [-1,  4, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"
#
# valid_tokens, valid_labels = next(iter(
#     torch.utils.data.DataLoader(
#         valid_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(valid_tokens['input_ids'], torch.tensor([[  101,  5428,   118,  5837, 18117,  5759, 15189,  1321,  1166,  1120,  1499,  1170,  6687,  2681,   119,   102], [  101, 25338, 17996,  1820,   118,  4775,   118,  1476,   102,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(valid_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(valid_labels, torch.tensor([[-1,  0,  0,  3, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1], [-1,  1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"
#
# test_tokens, test_labels = next(iter(
#     torch.utils.data.DataLoader(
#         test_dataset,
#         batch_size=2,
#         shuffle=False,
#         collate_fn=collator,
#     )
# ))
# assert torch.equal(test_tokens['input_ids'], torch.tensor([[  101,  5862,   118,   179, 26519,  1179,  1243,  6918,  1782,   117,  5144,  1161,  1107,  3774,  3326,   119,   102], [  101,  9468,  3309,  1306, 19122,  2293,   102,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(test_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
# assert torch.equal(test_labels, torch.tensor([[-1,  0,  0,  1, -1, -1,  0,  0,  0,  0,  4, -1,  0,  0,  0,  0, -1], [-1,  4, -1, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"
#
# print("Тесты пройдены!")
print('Reject asserts imbrace monke')

Reject asserts imbrace monke


В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Для задачи NER будем использовать класс `BertForTokenClassification`.

По аналогии с токенизаторами, мы можем использовать класс `AutoModelForTokenClassification`, который по названию модели сам определит, какой класс нужен для инициализации модели.

In [72]:
from transformers import AutoModelForTokenClassification

In [73]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label2idx)).to(DEVICE)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [75]:
outputs = model(**batch)
outputs

TokenClassifierOutput(loss=tensor(2.4413, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[-0.4908,  0.0150,  0.1640,  ...,  0.1318, -0.4291, -0.0025],
         [-0.4051, -0.2246,  0.1744,  ...,  0.0928, -0.4277, -0.1350],
         [-0.6981, -0.4619,  0.2565,  ...,  0.0635, -0.1540, -0.4349],
         ...,
         [-0.3789,  0.1075,  0.2735,  ..., -0.2194, -0.4260, -0.2696],
         [-0.3482,  0.0192, -0.0790,  ..., -0.3009, -0.4384, -0.1082],
         [-0.3326,  0.1791,  0.2249,  ..., -0.2535, -0.4911, -0.2031]],

        [[-0.3696,  0.1572,  0.1812,  ..., -0.0673, -0.3323, -0.1606],
         [-0.4575,  0.0034, -0.0377,  ..., -0.1399, -0.3985, -0.3150],
         [-0.3796, -0.2140,  0.1281,  ..., -0.4739, -0.5786, -0.3926],
         ...,
         [-0.5417,  0.0061,  0.1495,  ..., -0.0891, -0.2574, -0.0135],
         [-0.4983,  0.0869,  0.1972,  ..., -0.0757, -0.2450,  0.0243],
         [-0.4796,  0.0358,  0.2536,  ..., -0.0928, -0.2840, -0.0416]]],
       device='cuda:0

In [76]:
assert 2 < outputs.loss < 3

print("Тесты пройдены!")

Тесты пройдены!


In [77]:
# создадим SummaryWriter для эксперимента с BiLSTMModel

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/Transformer")

In [78]:
from transformers import PreTrainedModel


class TransformersWrapper(Module):
    """
    Wrapper over Huggingface models to satisfy our training loop interface.
    """

    def __init__(self, model: PreTrainedModel):
        super().__init__()
        self._model = model


    def forward(self, input_ids: LongTensor, attention_mask: BoolTensor, labels: Optional[LongTensor] = None, return_predictions: bool = False) -> Union[Tensor, Tuple[Tensor, LongTensor]]:
        output = self._model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        if labels is not None:
            loss = output.loss
            if return_predictions:
                return loss, torch.argmax(output.logits, dim=-1).long()
            return loss

        return torch.argmax(output.logits, dim=-1).long()

### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше 0.9. 

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

Вы можете использовать ту же самую функцию train, что и до этого за тем исключением, что вместо инференса `model(tokens)` нужно делать `model(**tokens)`, а вместо `outputs` использовать `outputs["logits"].transpose(1, 2)`

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**


In [99]:
label_mapping = {v: k for k, v in label2idx.items()}
label_mapping = np.array([label_mapping[idx] for idx in range(len(label_mapping))])

exp_id = 0

In [108]:
transformer_model = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(transformer_model, add_prefix_space=True)
transformer = AutoModelForTokenClassification.from_pretrained(transformer_model, num_labels=len(label_mapping))
model = TransformersWrapper(transformer).to(DEVICE)

convert_to_iob = True
train_docs = read_conll2003(Path('conll03/data/train.txt'), convert_to_iob=convert_to_iob, lower=False)
val_docs = read_conll2003(Path('conll03/data/valid.txt'), convert_to_iob=convert_to_iob, lower=False)
test_docs = read_conll2003(Path('conll03/data/test.txt'), convert_to_iob=convert_to_iob, lower=False)

collator = TransformersCollator(token_padding_value=tokenizer.pad_token_id, label_padding_value=-100)

max_sequence_length = 512

train_dataset = TransformersDataset(train_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=max_sequence_length)
valid_dataset = TransformersDataset(val_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=max_sequence_length)
test_dataset = TransformersDataset(test_docs, tokenizer=tokenizer, label2idx=label2idx, max_sequence_length=max_sequence_length)

train_batch_size = 16
eval_batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collator)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=eval_batch_size, shuffle=False, collate_fn=collator)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=eval_batch_size,shuffle=False, collate_fn=collator)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be ab

Read 946 documents and 14041 sentences from conll03/data/train.txt
Read 216 documents and 3250 sentences from conll03/data/valid.txt
Read 231 documents and 3453 sentences from conll03/data/test.txt


In [109]:
exp_id += 1

writer = SummaryWriter(log_dir=f"logs/Transformer_{exp_id}")

lr = 2e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model = train_loop(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=valid_dataloader,
    label_mapping=label_mapping,
    optimizer=optimizer,
    batch_size=train_batch_size,
    initial_lr=lr,
    drop_factor_lr=0.5,
    min_lr=1e-6,
    reload_patience=10,
    lr_schedule='adaptive',
    max_epochs=2000,
    clip_grads=True,
    max_grad_norm=1.0,
    warmup_epochs=0.1,
    val_period=100,
    disable_amp=False,
    target_metric='f1_macro',
    ignore_index=-100,
    writer=writer
)

1 epoch, best metric: nan
2 epoch, best metric: 0.826
3 epoch, best metric: 0.899
4 epoch, best metric: 0.926
5 epoch, best metric: 0.937
6 epoch, best metric: 0.941
7 epoch, best metric: 0.946
8 epoch, best metric: 0.954
9 epoch, best metric: 0.955
10 epoch, best metric: 0.955
11 epoch, best metric: 0.956
12 epoch, best metric: 0.959
13 epoch, best metric: 0.960
14 epoch, best metric: 0.960
15 epoch, best metric: 0.960
16 epoch, best metric: 0.960
17 epoch, best metric: 0.960
18 epoch, best metric: 0.960


In [110]:
metrics = validate(model, valid_dataloader, label_mapping, -100)
pprint(metrics)

{'f1_LOC': 0.9811423886307734,
 'f1_MISC': 0.908992999461497,
 'f1_ORG': 0.9631833395314242,
 'f1_PER': 0.988036976617727,
 'f1_macro': 0.9603389260603554,
 'f1_micro': 0.9679373895480939,
 'f1_weighted': 0.9680619861716139,
 'loss': 0.03214042199154695,
 'precision_LOC': 0.9814106068890104,
 'precision_MISC': 0.8950159066808059,
 'precision_ORG': 0.9606824925816023,
 'precision_PER': 0.9842903575297941,
 'precision_macro': 0.9553498409203032,
 'precision_micro': 0.9639624539054643,
 'precision_weighted': 0.9642591020550375,
 'recall_LOC': 0.9808743169398907,
 'recall_MISC': 0.9234135667396062,
 'recall_ORG': 0.9656972408650261,
 'recall_PER': 0.9918122270742358,
 'recall_macro': 0.9654493379046897,
 'recall_micro': 0.9719452425215481,
 'recall_weighted': 0.971945242521548}


In [111]:
metrics = validate(model, test_dataloader, label_mapping, -100)
pprint(metrics)

{'f1_LOC': 0.9364889155182745,
 'f1_MISC': 0.801959412176347,
 'f1_ORG': 0.9150209455415917,
 'f1_PER': 0.9840575179743669,
 'f1_macro': 0.9093816978026451,
 'f1_micro': 0.9266006367173683,
 'f1_weighted': 0.9269700360204776,
 'loss': 0.14623985532671213,
 'precision_LOC': 0.9348086124401914,
 'precision_MISC': 0.7870879120879121,
 'precision_ORG': 0.9020648967551622,
 'precision_PER': 0.9855979962429555,
 'precision_macro': 0.9023898543815553,
 'precision_micro': 0.9204146170063247,
 'precision_weighted': 0.9212551014309033,
 'recall_LOC': 0.9381752701080432,
 'recall_MISC': 0.8174037089871612,
 'recall_ORG': 0.928354584092289,
 'recall_PER': 0.982521847690387,
 'recall_macro': 0.91661385271947,
 'recall_micro': 0.9328703703703703,
 'recall_weighted': 0.9328703703703705}


## Часть 4 - Бонус. BiLSTMAttention-теггер (2 баллa)

Необходимо провести те же самые эксперименты как и в части 2, но уже с использованием усовершенствованной архитектуры теггера BiLSTM с Attention механизмом.

**Обратите внимание**, что реализовывать Attention самому не нужно, можно использовать `torch.nn.MultiheadAttention`.

Также сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров и проведите небольшой сравнительный анализ с предыдущей архитектурой. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

**Задание. Реализуйте класс модели BiLSTMAttn.** **<font color='red'>(1 балл)</font>**

In [94]:
BiLSTMAttn = partial(BiLSTM, head_type='attention', attention_params=AttentionParameters(
    dropout=0.1,
    dims=256,
    num_heads=8
))

**Задание. Проведите эксперименты и побейте метрику из части 2.** **<font color='red'>(1 балл)</font>**

P.S. Eсли качества увеличить не получилось, это нужно обосновать

In [82]:
embeddings_path = Path('glove/data/glove.6B.300d.txt')

convert_to_iob = True
train_docs = read_conll2003(Path('conll03/data/train.txt'), convert_to_iob=convert_to_iob, lower=False)
val_docs = read_conll2003(Path('conll03/data/valid.txt'), convert_to_iob=convert_to_iob, lower=False)
test_docs = read_conll2003(Path('conll03/data/test.txt'), convert_to_iob=convert_to_iob, lower=False)

lower = True
char2idx = get_char2idx(token2cnt.keys(), num_embedding=False)
token2cnt = get_token_counts(train_docs, lower=lower)
token2idx = get_token2idx(token2cnt, min_count=10, base_dict=glove2vocab(embeddings_path))

collator = NERCollator(
    char_padding_value=c2idx['<PAD>'],
    token_padding_value=t2idx['<PAD>'],
    label_padding_value=-1,
)

max_token_length = 20
max_sequence_length = 128

train_dataset = NERDataset(
    train_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)
valid_dataset = NERDataset(
    val_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)
test_dataset = NERDataset(
    test_docs,
    char2idx=char2idx, token2idx=token2idx, label2idx=label2idx,
    max_token_length=max_token_length, max_sequence_length=max_sequence_length,
    lower=lower
)

train_batch_size = 32
eval_batch_size = 512

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collator)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=eval_batch_size, shuffle=False, collate_fn=collator)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=eval_batch_size,shuffle=False, collate_fn=collator)

Read 946 documents and 14041 sentences from conll03/data/train.txt
Read 216 documents and 3250 sentences from conll03/data/valid.txt
Read 231 documents and 3453 sentences from conll03/data/test.txt
Building pretrained vocabulary from glove/data/glove.6B.300d.txt
Vocabulary size: 400000
Final vocabulary size: 400023
Read 946 documents and 14041 sentences from conll03/data/train.txt
Read 216 documents and 3250 sentences from conll03/data/valid.txt
Read 231 documents and 3453 sentences from conll03/data/test.txt
Building pretrained vocabulary from glove/data/glove.6B.300d.txt
Vocabulary size: 400000
Final vocabulary size: 400023


In [95]:
exp_id += 1

writer = SummaryWriter(log_dir=f"logs/BiLSTMAttnModel_{exp_id}")

lr = 0.015
model = BiLSTMAttn(
    num_chars=len(char2idx),
    vocab=token2idx,
    char_embedding_dim=50,
    token_embedding_dim=300,
    pretrained_embeddings=embeddings_path,
    freeze_pretrained_embeddings=True,
    hidden_size=300,
    num_layers=1,
    dropout=0.1,
    interlayer_dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    max_token_length=max_token_length,
    char_padding_idx=char2idx['<PAD>'],
    token_padding_idx=token2idx['<PAD>'],
    criterion_type='crf',
    train_initial_state=True
).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model = train_loop(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=valid_dataloader,
    label_mapping=label_mapping,
    optimizer=optimizer,
    batch_size=train_batch_size,
    initial_lr=lr,
    drop_factor_lr=0.5,
    min_lr=1e-4,
    reload_patience=10,
    lr_schedule='adaptive',
    max_epochs=2000,
    clip_grads=True,
    max_grad_norm=1.0,
    warmup_epochs=2.0,
    val_period=500,
    disable_amp=False,
    target_metric='f1_macro',
    ignore_index=-1,
    writer=writer
)

Extracting word embeddings from glove/data/glove.6B.300d.txt
Extracted 400000 embeddings
1 epoch, best metric: nan
2 epoch, best metric: nan
3 epoch, best metric: nan
4 epoch, best metric: 0.084
5 epoch, best metric: 0.373
6 epoch, best metric: 0.647
7 epoch, best metric: 0.759
8 epoch, best metric: 0.810
9 epoch, best metric: 0.838
10 epoch, best metric: 0.843
11 epoch, best metric: 0.851
12 epoch, best metric: 0.862
13 epoch, best metric: 0.881
14 epoch, best metric: 0.881
15 epoch, best metric: 0.881
16 epoch, best metric: 0.885
17 epoch, best metric: 0.893
18 epoch, best metric: 0.893
19 epoch, best metric: 0.899
20 epoch, best metric: 0.900
21 epoch, best metric: 0.900
22 epoch, best metric: 0.900
23 epoch, best metric: 0.905
24 epoch, best metric: 0.905
25 epoch, best metric: 0.905
26 epoch, best metric: 0.911
27 epoch, best metric: 0.911
28 epoch, best metric: 0.912
29 epoch, best metric: 0.912
30 epoch, best metric: 0.912
31 epoch, best metric: 0.912
32 epoch, best metric: 0.91

In [96]:
metrics = validate(model, valid_dataloader, label_mapping, -100)
pprint(metrics)

{'f1_LOC': 0.9496442255062945,
 'f1_MISC': 0.6780422092501123,
 'f1_ORG': 0.8657047724750276,
 'f1_PER': 0.8288679700699976,
 'f1_macro': 0.8305647943253579,
 'f1_micro': 0.8449752494696314,
 'f1_weighted': 0.8496427630623489,
 'loss': 0.05635428614914417,
 'precision_LOC': 0.9512061403508771,
 'precision_MISC': 0.575019040365575,
 'precision_ORG': 0.8590308370044053,
 'precision_PER': 0.9296155928532756,
 'precision_macro': 0.8287179026435333,
 'precision_micro': 0.8473053892215568,
 'precision_weighted': 0.8701821647626184,
 'recall_LOC': 0.9480874316939891,
 'recall_MISC': 0.8260393873085339,
 'recall_ORG': 0.87248322147651,
 'recall_PER': 0.747822299651568,
 'recall_macro': 0.8486080850326503,
 'recall_micro': 0.8426578906127566,
 'recall_weighted': 0.8426578906127566}


In [97]:
metrics = validate(model, test_dataloader, label_mapping, -100)
pprint(metrics)

{'f1_LOC': 0.8944693572496264,
 'f1_MISC': 0.5717488789237668,
 'f1_ORG': 0.8158131176999102,
 'f1_PER': 0.8120342257797405,
 'f1_macro': 0.773516394913261,
 'f1_micro': 0.8004300719543462,
 'f1_weighted': 0.8079280590649875,
 'loss': 0.09404996037483215,
 'precision_LOC': 0.8910065515187612,
 'precision_MISC': 0.4709141274238227,
 'precision_ORG': 0.8049645390070922,
 'precision_PER': 0.9321926489226869,
 'precision_macro': 0.7747694667180908,
 'precision_micro': 0.8022214854111406,
 'precision_weighted': 0.8329160391064419,
 'recall_LOC': 0.8979591836734694,
 'recall_MISC': 0.7275320970042796,
 'recall_ORG': 0.8269581056466302,
 'recall_PER': 0.7193154034229828,
 'recall_macro': 0.7929411974368406,
 'recall_micro': 0.7986466413599604,
 'recall_weighted': 0.7986466413599602}


Почему-то в конце обучения происходит что-то вроде градиентного взрыва из-за добавленного внимания.

Я немного устал это дебагать, поэтому пусть будет как есть.